In [64]:

import pandas as pd
import re
import os

def parse_training_results(file_path):
    """Parse training results from text file into a pandas DataFrame"""
    
    # Read the file
    with open(file_path, 'r') as f:
        content = f.read()
    
    # Initialize lists to store data
    epochs = []
    times = []
    train_losses = []
    train_acc_top1 = []
    train_acc_top5 = []
    test_losses = []
    test_acc_top1 = []
    test_acc_top5 = []
    
    # Regular expression to match each epoch line
    pattern = r'\[Epoch (\d+)\] Time: ([\d.]+)s.*?Train.*?Loss: ([\d.]+).*?Top1: ([\d.]+)%.*?Top5: ([\d.]+)%.*?Test.*?Loss: ([\d.]+).*?Top1: ([\d.]+)%.*?Top5: ([\d.]+)%'
    
    # Find all matches
    matches = re.findall(pattern, content)
    
    for match in matches:
        epochs.append(int(match[0]))
        times.append(float(match[1]))
        train_losses.append(float(match[2]))
        train_acc_top1.append(float(match[3]))
        train_acc_top5.append(float(match[4]))
        test_losses.append(float(match[5]))
        test_acc_top1.append(float(match[6]))
        test_acc_top5.append(float(match[7]))
    
    # Create DataFrame
    df = pd.DataFrame({
        'epoch': epochs,
        'time': times,
        'train_loss': train_losses,
        'train_accuracy_top1': train_acc_top1,
        'train_accuracy_top5': train_acc_top5,
        'test_loss': test_losses,
        'test_accuracy_top1': test_acc_top1,
        'test_accuracy_top5': test_acc_top5
    })
    
    return df


def n_last_average_stats(df, last_n_epochs=5):
    """Calculate average stats over the last N epochs"""
    if len(df) < last_n_epochs:
        last_n_epochs = len(df)
    
    return {
        "ave_train_loss": df['train_loss'].tail(last_n_epochs).mean(),
        "ave_test_loss": df['test_loss'].tail(last_n_epochs).mean(),
        "ave_train_accuracy_top1": df['train_accuracy_top1'].tail(last_n_epochs).mean(),
        "ave_train_accuracy_top5": df['train_accuracy_top5'].tail(last_n_epochs).mean(),
        "ave_test_accuracy_top1": df['test_accuracy_top1'].tail(last_n_epochs).mean(),
        "ave_test_accuracy_top5": df['test_accuracy_top5'].tail(last_n_epochs).mean()
    }

def best_stats(df):
    """Calculate best accuracy achieved during training"""
    return {
        "best_train_accuracy_top1": df['train_accuracy_top1'].max(),
        "best_train_accuracy_top5": df['train_accuracy_top5'].max(),
        "best_test_accuracy_top1": df['test_accuracy_top1'].max(),
        "best_test_accuracy_top5": df['test_accuracy_top5'].max()
    }

def last_epoch_stats(df):
    """Get stats from the last epoch"""
    last_row = df.iloc[-1]
    return {
        "last_epoch_train_loss": last_row['train_loss'],
        "last_epoch_test_loss": last_row['test_loss'],
        "last_epoch_train_accuracy_top1": last_row['train_accuracy_top1'],
        "last_epoch_train_accuracy_top5": last_row['train_accuracy_top5'],
        "last_epoch_test_accuracy_top1": last_row['test_accuracy_top1'],
        "last_epoch_test_accuracy_top5": last_row['test_accuracy_top5']
    }

def parse_params_gflops(file_dir):
    """Parse training results from text file into a pandas DataFrame"""

    train_path = os.path.join(file_dir, "train_eval_results.txt")
    args_path = os.path.join(file_dir, "args.txt")
    
    # Read the file
    with open(train_path, 'r') as f:
        train_content = f.readline()

    with open(args_path, 'r') as f:
        args_content = f.readlines()

    total_params = args_content[-2].strip().split(": ")[1]
    trainable_params = args_content[-1].strip().split(": ")[1]

    gflops = re.search(r'GFLOPs: ([\d.]+)', train_content).group(1)

    return {
        "total_params": int(total_params),
        "trainable_params": int(trainable_params),
        "gflops": float(gflops)
    }



In [65]:
import sys 
import os
import torch 
import torch.nn as nn

curr_dir = os.getcwd()
root_dir = os.path.abspath(os.path.join(curr_dir, '..', '..'))

print("Current Directory:", curr_dir)
print("Root Directory:", root_dir)


Current Directory: /Users/mingikang/Developer/Convolutional-Nearest-Neighbor-Attention
Root Directory: /Users/mingikang


# 1. Baseline Test

In [66]:
save_path =  os.path.join(curr_dir, "Final_csv", "baseline_test.csv")
save_dir = os.path.dirname(save_path)




In [67]:
datasets = ["ViT-Tiny-CIFAR10", "ViT-Tiny-CIFAR100"]
layers = ["SparseAttention_Local_NH1_CW49_s42", 
          "SparseAttention_Strided_NH1_CW14_s42",
          "Attention_NH1_s42", 
          "LocalAttention_NH1_s42", 
          "ConvNNAttention_K9_s42",
          "ConvNNAttention_K9_N32_random_s42", 
          "ConvNNAttention_K9_N32_spatial_s42"
]

rows = [] 
for ds in datasets:
    for layer in layers: 
        if "SparseAttention" in layer:
            results_path = os.path.join(curr_dir, "Final_Output", "Baseline_Test_Sparse---/")
        elif layer == "ConvNNAttention_K9_s42":
            results_path = os.path.join(curr_dir, "Final_Output", "K_test_correct---")
        elif layer == "ConvNNAttention_K9_N32_random_s42" or layer == "ConvNNAttention_K9_N32_spatial_s42":
            results_path = os.path.join(curr_dir, "Final_Output", "N_test_correct---")
        else: 
            results_path = os.path.join(curr_dir, "Final_Output", "Baseline_Test---")


        dir_path = os.path.join(results_path, ds, layer) 
        txt_path = os.path.join(dir_path, "train_eval_results.txt")

        df = parse_training_results(txt_path)
        ave_stats = n_last_average_stats(df, last_n_epochs=5)
        best_acc = best_stats(df) 
        last_stats = last_epoch_stats(df)
        params_gflops = parse_params_gflops(dir_path)

        row = {
            "dataset": ds,
            "layer": layer,
            **ave_stats,
            **best_acc,
            **last_stats,
            **params_gflops
        }
        rows.append(row)
        
df_results = pd.DataFrame(rows)
df_results.to_csv(save_path, index=False)


# 2. K Test

In [78]:
save_path =  os.path.join(curr_dir, "Final_csv", "k_test.csv")
save_dir = os.path.dirname(save_path)


results_path = os.path.join(curr_dir, "Final_Output", "K_test_correct---/")
print("Results Path:", results_path)

Results Path: /Users/mingikang/Developer/Convolutional-Nearest-Neighbor-Attention/Final_Output/K_test_correct---/


In [79]:
# Categories 
datasets = ["ViT-Tiny-CIFAR10", "ViT-Tiny-CIFAR100"]
Ks = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
layers = ["ConvNNAttention", "KvtAttention", "ConvNNAttentionRandom"]

rows = [] 

for ds in datasets: 
    for layer in layers: 
        for k in Ks:
            if layer == "ConvNNAttention": 
                dir_path = os.path.join(results_path, ds, f"ConvNNAttention_K{k}_s42")
            elif layer == "KvtAttention":
                dir_path = os.path.join(results_path, ds, f"KvtAttention_K{k}_s42")
            elif layer == "ConvNNAttentionRandom":
                dir_path = os.path.join(results_path, ds, f"ConvNNAttention_Random_K{k}_NS32_s42")
                
            txt_path = os.path.join(dir_path, "train_eval_results.txt")

            df = parse_training_results(txt_path)
            ave_stats = n_last_average_stats(df, last_n_epochs=5)
            best_acc = best_stats(df)
            last_stats = last_epoch_stats(df)
            params_gflops = parse_params_gflops(dir_path)


            if layer == "ConvNNAttention" or layer == "ConvNNAttentionRandom":
                layer_name = "ConvNNAttention"
            else:
                layer_name = layer

            if layer == "ConvNNAttentionRandom":
                layer_type = "random"
            elif layer == "ConvNNAttention":
                layer_type = "all"
            else: 
                layer_type = "N/A"
                
            row = {
                "dataset": ds,
                "layer": layer_name,
                "type": layer_type, 
                "K": k, 
                **ave_stats,
                **best_acc,
                **last_stats,
                **params_gflops
            }
            rows.append(row)

    dir_path = os.path.join(results_path, ds, f"KvtAttention_K100_s42")
    txt_path = os.path.join(dir_path, "train_eval_results.txt")
    df = parse_training_results(txt_path)
    ave_stats = n_last_average_stats(df, last_n_epochs=5)
    best_acc = best_stats(df)
    last_stats = last_epoch_stats(df)
    params_gflops = parse_params_gflops(dir_path)
    row = {
        "dataset": ds,
        "layer": "KvtAttention",
        "type": "N/A", 
        "K": 100, 
        **ave_stats,
        **best_acc,
        **last_stats,
        **params_gflops
    }
    rows.append(row)

    attention_baseline_dir = os.path.join(curr_dir, "Final_Output", "Baseline_Test---/")
    dir_path = os.path.join(attention_baseline_dir, ds, "Attention_NH1_s42")
    txt_path = os.path.join(dir_path, "train_eval_results.txt")
    df = parse_training_results(txt_path)
    ave_stats = n_last_average_stats(df, last_n_epochs=5)
    best_acc = best_stats(df)
    last_stats = last_epoch_stats(df)
    params_gflops = parse_params_gflops(dir_path)
    row = {
        "dataset": ds,
        "layer": "Attention",
        "type": "N/A", 
        "K": -1, 
        **ave_stats,
        **best_acc,
        **last_stats,
        **params_gflops
    }
    rows.append(row)


df_results = pd.DataFrame(rows)
df_results.to_csv(save_path, index=False)


In [70]:
df_results.head()

,dataset,layer,type,K,ave_train_loss,ave_test_loss,ave_train_accuracy_top1,ave_train_accuracy_top5,ave_test_accuracy_top1,ave_test_accuracy_top5,...,best_test_accuracy_top5,last_epoch_train_loss,last_epoch_test_loss,last_epoch_train_accuracy_top1,last_epoch_train_accuracy_top5,last_epoch_test_accuracy_top1,last_epoch_test_accuracy_top5,total_params,trainable_params,gflops
0,ViT-Tiny-CIFAR10,ConvNNAttention,all,1,0.484201,0.853181,82.45680,99.53148,73.44532,98.26954,...,98.4375,0.480270,0.846335,82.5136,99.5635,73.5352,98.1934,5481610,5481610,2.329554
1,ViT-Tiny-CIFAR10,ConvNNAttention,all,2,0.057783,1.677184,97.96586,99.99880,73.24414,97.86718,...,98.2910,0.054208,1.710143,98.0397,100.0000,73.3496,97.9590,5483914,5483914,2.330008
2,ViT-Tiny-CIFAR10,ConvNNAttention,all,3,0.039318,1.808193,98.61320,99.99920,73.35154,97.88476,...,98.2129,0.037606,1.804209,98.6368,100.0000,73.4570,97.8711,5486218,5486218,2.330462
3,ViT-Tiny-CIFAR10,ConvNNAttention,all,4,0.037307,1.880055,98.70406,99.99960,72.72852,97.89844,...,98.2129,0.036487,1.811751,98.7424,100.0000,73.5742,97.9590,5488522,5488522,2.330916
4,ViT-Tiny-CIFAR10,ConvNNAttention,all,5,0.032257,1.890058,98.88218,99.99880,73.23634,97.89260,...,98.2324,0.033314,1.822230,98.8656,99.9980,73.5547,97.9199,5490826,5490826,2.331369


# 3. N Test

In [71]:
save_path =  os.path.join(curr_dir, "Final_csv", "n_test.csv")
save_dir = os.path.dirname(save_path)


results_path = os.path.join(curr_dir, "Final_Output", "N_test_correct---/")
print("Results Path:", results_path)

Results Path: /Users/mingikang/Developer/Convolutional-Nearest-Neighbor-Attention/Final_Output/N_test_correct---/


In [72]:

# Categories 
datasets = ["ViT-Tiny-CIFAR10", "ViT-Tiny-CIFAR100"]
type = ["random", "spatial"]
Ns = ["16", "32", "48", "64", "80", "96", "112", "128", "144", "160", "176", "192"]
rows = []

layers = ["Attention_NH1_s42", 
          "ConvNNAttention_K9_s42",
]
for ds in datasets:
    for t in type:
        for n in Ns:
            dir_path = os.path.join(results_path, ds, f"ConvNNAttention_K9_N{n}_{t}_s42")
            txt_path = os.path.join(dir_path, "train_eval_results.txt")

            df = parse_training_results(txt_path)
            avg_stats = n_last_average_stats(df, last_n_epochs=5)
            best_acc = best_stats(df)
            last_stats = last_epoch_stats(df)
            params_gflops = parse_params_gflops(dir_path)

            row = {
                "dataset": ds,
                "layer": "ConvNNAttention",
                "type": t,
                "N": int(n), 
                **avg_stats, 
                **best_acc,
                **last_stats,
                **params_gflops
            }
            rows.append(row)
    
    for layer in layers:
        if layer == "Attention_NH1_s42":
            results_path1 = os.path.join(curr_dir, "Final_Output", "Baseline_Test---/")
        elif layer == "ConvNNAttention_K9_s42":
            results_path1 = os.path.join(curr_dir, "Final_Output", "K_test_correct---")

        dir_path = os.path.join(results_path1, ds, layer)
        txt_path = os.path.join(dir_path, "train_eval_results.txt")

        df = parse_training_results(txt_path)
        avg_stats = n_last_average_stats(df, last_n_epochs=5)
        best_acc = best_stats(df)
        last_stats = last_epoch_stats(df)
        params_gflops = parse_params_gflops(dir_path)

        if layer == "Attention_NH1_s42":
            row = {
                "dataset": ds,
                "layer": "Attention",
                "type": "N/A",
                "N": -1,
                **avg_stats,
                **best_acc,
                **last_stats,
                **params_gflops
            }
        else: 
                
            row = {
                "dataset": ds,
                "layer": "ConvNNAttention",
                "type": "all",
                "N": -1,
                **avg_stats,
                **best_acc,
                **last_stats,
                **params_gflops
            }
        rows.append(row)

df_results = pd.DataFrame(rows)
df_results.to_csv(save_path, index=False)

In [73]:
df_results.head()


,dataset,layer,type,N,ave_train_loss,ave_test_loss,ave_train_accuracy_top1,ave_train_accuracy_top5,ave_test_accuracy_top1,ave_test_accuracy_top5,...,best_test_accuracy_top5,last_epoch_train_loss,last_epoch_test_loss,last_epoch_train_accuracy_top1,last_epoch_train_accuracy_top5,last_epoch_test_accuracy_top1,last_epoch_test_accuracy_top5,total_params,trainable_params,gflops
0,ViT-Tiny-CIFAR10,ConvNNAttention,random,16,0.268761,0.726744,90.15042,99.92746,78.81056,98.84182,...,99.0137,0.260786,0.735397,90.4974,99.9342,79.0430,98.8770,5500042,5500042,2.008740
1,ViT-Tiny-CIFAR10,ConvNNAttention,random,32,0.182491,0.772499,93.36926,99.96970,79.95508,98.94532,...,99.0820,0.179283,0.779419,93.4805,99.9661,79.8242,99.0039,5500042,5500042,2.037420
2,ViT-Tiny-CIFAR10,ConvNNAttention,random,48,0.131127,0.861982,95.26020,99.99040,79.76368,98.92382,...,99.0234,0.126762,0.866322,95.4676,99.9880,79.6973,98.8770,5500042,5500042,2.066101
3,ViT-Tiny-CIFAR10,ConvNNAttention,random,64,0.117298,0.948186,95.81624,99.99160,78.68166,98.66602,...,98.8574,0.115586,0.949326,95.8857,99.9940,78.7500,98.7793,5500042,5500042,2.094781
4,ViT-Tiny-CIFAR10,ConvNNAttention,random,80,0.099740,1.005526,96.40392,99.99600,78.68946,98.60352,...,98.8770,0.095422,0.973670,96.5354,99.9920,79.2578,98.7695,5500042,5500042,2.123461


# 6. Loss CSV

In [74]:
save_path =  os.path.join(curr_dir, "Final_csv", "loss_test.csv")
save_dir = os.path.dirname(save_path)




In [75]:
datasets = ["ViT-Tiny-CIFAR10", "ViT-Tiny-CIFAR100"]
layers = ["SparseAttention_Local_NH1_CW49_s42", 
          "SparseAttention_Strided_NH1_CW14_s42",
          "Attention_NH1_s42", 
          "LocalAttention_NH1_s42", 
          "ConvNNAttention_K9_s42",
          "ConvNNAttention_K9_N32_random_s42", 
          "ConvNNAttention_K9_N32_spatial_s42"
]

rows = [] 
for ds in datasets:
    for layer in layers: 
        if "SparseAttention" in layer:
            results_path = os.path.join(curr_dir, "Final_Output", "Baseline_Test_Sparse---/")
        elif layer == "ConvNNAttention_K9_s42":
            results_path = os.path.join(curr_dir, "Final_Output", "K_test_correct---")
        elif layer == "ConvNNAttention_K9_N32_random_s42" or layer == "ConvNNAttention_K9_N32_spatial_s42":
            results_path = os.path.join(curr_dir, "Final_Output", "N_test_correct---")
        else: 
            results_path = os.path.join(curr_dir, "Final_Output", "Baseline_Test---")


        dir_path = os.path.join(results_path, ds, layer) 
        txt_path = os.path.join(dir_path, "train_eval_results.txt")

        df = parse_training_results(txt_path)

        length = len(df['epoch'])
        ds_list = [ds] * length
        layer_list = [layer] * length

        for l in range(length):
            row = {
                "dataset": ds,
                "layer": layer,
                "epoch": df['epoch'][l], 
                "train_loss": df['train_loss'][l],
                "test_loss": df['test_loss'][l],
            }
        
            rows.append(row)
        
df_results = pd.DataFrame(rows)
df_results.to_csv(save_path, index=False)


In [76]:
df_results.head()


,dataset,layer,epoch,train_loss,test_loss
0,ViT-Tiny-CIFAR10,SparseAttention_Local_NH1_CW49_s42,1,2.031195,1.913442
1,ViT-Tiny-CIFAR10,SparseAttention_Local_NH1_CW49_s42,2,1.846178,1.768176
2,ViT-Tiny-CIFAR10,SparseAttention_Local_NH1_CW49_s42,3,1.706714,1.598489
3,ViT-Tiny-CIFAR10,SparseAttention_Local_NH1_CW49_s42,4,1.585187,1.490248
4,ViT-Tiny-CIFAR10,SparseAttention_Local_NH1_CW49_s42,5,1.451314,1.372131
